## Data validation:
This notebook contains the code for performing data validation on the Mercari Dataset. The ideal choice for performing data validation in python is using `tensorflow data validation`. Since, it is not compatible with our python environment, we can use the following packages as alternatives:
- great expectations
- pandas-profiling
- pydantic

In [1]:
# import essentials
import numpy
import pandas as pd
import pandera as pa
import matplotlib.pyplot as plt
import great_expectations as gx
from great_expectations.checkpoint import Checkpoint
%matplotlib inline

## Great expectations:

Create data context

In [2]:
context=gx.get_context()

Get the data from cloud storage

In [3]:
gcp_url = 'https://storage.googleapis.com/price_alchemy/Data/data.csv'

df = pd.read_csv(gcp_url)

In [4]:
# add data sources
DS_NAME='train_file'
datasource=context.sources.add_pandas(name=DS_NAME)

# adding csv asset
asset_name='mercari_training'
asset= datasource.add_dataframe_asset(asset_name)

# build batch request
batch_request= asset.build_batch_request(dataframe= df)

Add expectation suite

In [5]:
context.add_or_update_expectation_suite('mercari_first_expectation_suite')

{
  "expectation_suite_name": "mercari_first_expectation_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.10"
  }
}

In [6]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name='mercari_first_expectation_suite',
)

What does the data look like?

In [7]:
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,created_at,last_updated_at
0,793697,Plaid Vest,2,Women/Coats & Jackets/Vest,Old Navy,11.0,1,Green and blue. Very thick and soft! Perfect f...,2022-01-01 00:00:00,2022-01-01 00:00:00
1,402094,Women's Sperrys,3,Women/Shoes/Loafers & Slip-Ons,Sperrys,21.0,0,EUC,2022-01-01 00:01:00,2022-01-01 00:01:00
2,522439,Grey sweater dress,1,Women/Dresses/Other,Fashion Nova,20.0,1,This is a heather grey sweater dress from fash...,2022-01-01 00:01:00,2022-01-01 00:01:00
3,214455,Tory Burch 'Perry' Leather Wallet,3,Women/Women's Accessories/Wallets,Tory Burch,91.0,0,Tory Burch 'Perry' Leather Zip Continental Wal...,2022-01-01 00:03:00,2022-01-01 00:03:00
4,902755,Fujifilm Rainbow Instax Film,1,Electronics/Cameras & Photography/Film Photogr...,Fuji,14.0,0,No description yet,2022-01-01 00:05:00,2022-01-01 00:05:00


Let's create some expectations

In [8]:
# column should exist
validator.expect_column_to_exist("name")
validator.expect_column_to_exist("price")
validator.expect_column_to_exist("item_condition_id")
validator.expect_column_to_exist("category_name")
validator.expect_column_to_exist("brand_name")
validator.expect_column_to_exist("shipping")
validator.expect_column_to_exist("item_description")
validator.expect_column_to_exist("created_at")
validator.expect_column_to_exist("last_updated_at")

# not null expectations
validator.expect_column_values_to_not_be_null("name")
validator.expect_column_values_to_not_be_null("price")
validator.expect_column_values_to_not_be_null("item_condition_id")
validator.expect_column_values_to_not_be_null("shipping")
validator.expect_column_values_to_not_be_null("created_at")
validator.expect_column_values_to_not_be_null("last_updated_at")
validator.expect_column_values_to_not_be_null("item_description", mostly=0.95)
validator.expect_column_values_to_not_be_null("category_name", mostly=.95)
validator.expect_column_values_to_not_be_null("brand_name", mostly=0.5)

# value expectations
# validator.expect_column_values_to_be_between(
#     "price", min_value=0, max_value=2000)

validator.expect_column_max_to_be_between(
    "price", min_value=1000, max_value=2500)

validator.expect_column_distinct_values_to_be_in_set(
        "shipping",
        [0,1])

validator.expect_column_distinct_values_to_be_in_set(
        "item_condition_id",
        [1,2,3,4,5])

#  distribution expectations
validator.expect_column_stdev_to_be_between(
'price', min_value=30, max_value=50)

validator.expect_column_mean_to_be_between(
'price', min_value=20, max_value=30)

validator.expect_column_value_z_scores_to_be_less_than(
'price', threshold=3, mostly=.9, double_sided=False)

# regex expectations
# should not be urls
validator.expect_column_values_to_not_match_regex(
'name', regex='https?:\/\/.*[\r\n]*')

validator.expect_column_values_to_not_match_regex(
'brand_name', regex='https?:\/\/.*[\r\n]*')

validator.expect_column_values_to_not_match_regex(
'category_name', regex='https?:\/\/.*[\r\n]*')

# date columns
validator.expect_column_values_to_be_dateutil_parseable('created_at')
validator.expect_column_values_to_be_dateutil_parseable('last_updated_at')

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 972406,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Let's run the data assistant 

In [9]:
# exclude_cols_names=['train_id']

In [10]:
# data_assistant_result = context.assistants.missingness.run(
#     validator=validator,
#     exclude_column_names=exclude_cols_names,
# )

Save expectations

In [11]:
# validator.expectation_suite= data_assistant_result.get_expectation_suite(
#     expectation_suite_name='mercari_first_expectation_suite'
# )
validator.save_expectation_suite(discard_failed_expectations=False)

In [12]:
checkpoint = context.add_or_update_checkpoint(
    name="checkpoint_v4",
    validator=validator,
)

Run expectations

In [13]:
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/84 [00:00<?, ?it/s]

In [14]:
# assert checkpoint_result["success"] is True

View results 

In [15]:
# data_assistant_result.show_expectations_by_expectation_type()

Let's view the data doc

In [16]:
context.view_validation_result(checkpoint_result)

## Pandera:

Load dataframe

In [17]:
gcp_url = 'https://storage.googleapis.com/price_alchemy/Data/data.csv'

df = pd.read_csv(gcp_url)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972406 entries, 0 to 972405
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   train_id           972406 non-null  int64  
 1   name               972406 non-null  object 
 2   item_condition_id  972406 non-null  int64  
 3   category_name      967865 non-null  object 
 4   brand_name         534339 non-null  object 
 5   price              972406 non-null  float64
 6   shipping           972406 non-null  int64  
 7   item_description   972403 non-null  object 
 8   created_at         972406 non-null  object 
 9   last_updated_at    972406 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 74.2+ MB


In [19]:
df.head(10)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,created_at,last_updated_at
0,793697,Plaid Vest,2,Women/Coats & Jackets/Vest,Old Navy,11.0,1,Green and blue. Very thick and soft! Perfect f...,2022-01-01 00:00:00,2022-01-01 00:00:00
1,402094,Women's Sperrys,3,Women/Shoes/Loafers & Slip-Ons,Sperrys,21.0,0,EUC,2022-01-01 00:01:00,2022-01-01 00:01:00
2,522439,Grey sweater dress,1,Women/Dresses/Other,Fashion Nova,20.0,1,This is a heather grey sweater dress from fash...,2022-01-01 00:01:00,2022-01-01 00:01:00
3,214455,Tory Burch 'Perry' Leather Wallet,3,Women/Women's Accessories/Wallets,Tory Burch,91.0,0,Tory Burch 'Perry' Leather Zip Continental Wal...,2022-01-01 00:03:00,2022-01-01 00:03:00
4,902755,Fujifilm Rainbow Instax Film,1,Electronics/Cameras & Photography/Film Photogr...,Fuji,14.0,0,No description yet,2022-01-01 00:05:00,2022-01-01 00:05:00
5,1284875,Tartelette,3,Beauty/Makeup/Makeup Palettes,Tarte,25.0,1,Used. It looks new,2022-01-01 00:06:00,2022-01-01 00:06:00
6,973923,New Electric Warmer Faith Hope Love &,1,Beauty/Fragrance/Candles & Home Scents,NaN,20.0,0,My prices are fair and firm ☆Full size warmers...,2022-01-01 00:14:00,2022-01-01 00:14:00
7,495229,eroshiyda's Pumpkin Pie Lip Scrub Set,1,Beauty/Skin Care/Lips,NaN,6.0,1,"This is allergen-free cruelty-free, dye-free, ...",2022-01-01 00:15:00,2022-01-01 00:15:00
8,829109,AEO Ripped Jeans,3,"Women/Jeans/Slim, Skinny",American Eagle,21.0,0,Size 8,2022-01-01 00:19:00,2022-01-01 00:19:00
9,1013975,Air Jordan 4,2,Kids/Boys (4+)/Shoes,Nike,119.0,0,J4 White Cement Like new condition With origin...,2022-01-01 00:25:00,2022-01-01 00:25:00


Let's  use pandera to declare the schema of the dataframe

In [20]:
schema= pa.DataFrameSchema({
    'train_id':pa.Column(int),
    'name': pa.Column(str), 
    'item_condition_id': pa.Column(int, checks=pa.Check.isin([1,2,3,4,5])),
    'category_name':pa.Column(str, nullable=True),
    'brand_name':pa.Column(str, nullable=True),
    'price':pa.Column(float, checks=pa.Check(lambda x: x>=0)), 
    'shipping':pa.Column(int, checks=pa.Check.isin([0,1])),
    'item_description':pa.Column(str, nullable=True),
    'created_at':pa.Column(str),
    'last_updated_at':pa.Column(str)
}, 
unique=['train_id','name'])

Validate the schema of the dataframe

In [21]:
try :
    schema(df)
    validated= True
except:
    validated= False

validated

True

In [22]:
if validated:
    print(schema(df))

        train_id                               name  item_condition_id  \
0         793697                         Plaid Vest                  2   
1         402094                    Women's Sperrys                  3   
2         522439                 Grey sweater dress                  1   
3         214455  Tory Burch 'Perry' Leather Wallet                  3   
4         902755       Fujifilm Rainbow Instax Film                  1   
...          ...                                ...                ...   
972401    700766           LF Floral Hooded Flannel                  3   
972402   1277823       Rae Dunn FAMILY & FEAST Mugs                  1   
972403   1446057             ‼️LAST CHANCE TO GET‼️                  3   
972404    222392                 ⭐️OPI - Race Red⭐️                  1   
972405    151885                 Boho Cardigan Vest                  2   

                                            category_name       brand_name  \
0                              Wo